In [165]:
# load packages
import numpy as np
import scipy as sp
import pandas as pd
from scipy import optimize as opt
from scipy.integrate import odeint 
from scipy.optimize import minimize

Now consider parameter learning for the SuEIR model. Given the model parameters $\boldsymbol{\theta}$ and initial quantities $S_0, E_0, I_0$, and $R_0$, we can compute the number of individuals in each group (i.e., $S, E, I$, and $R$ ) at time $t$, denoted by $\widehat{S}_t, \widehat{E}_t, \widehat{I}_t$ and $\widehat{R}_t$, via applying  numerical ODE solvers onto the ODE. Then we propose to learn the model parameter $\widehat{\boldsymbol{\theta}}=(\widehat{\beta}, \widehat{\sigma}, \widehat{\gamma}, \widehat{\mu})$ by minimizing the following logarithmic-type mean square error (MSE).

Where $\mathbf{I}=\left\{I_t\right\}_{t=1}^T, \mathbf{R}=\left\{R_t\right\}_{t=1}^T$ with $I_t$ and $R_t$ denote the reported numbers of infected cases and removed cases (including both recovered cases and fatality cases) at time $t$ (i.e., date), and $p$ is the smoothing parameter used to ensure numerical stability. Note that given $S_0, E_0, I_0$ and $R_0, \widehat{I}_t$ and $\widehat{R}_t$ can be described as differentiable functions of the parameter $\boldsymbol{\theta}$. Then the model parameter $\widehat{\boldsymbol{\theta}}=\operatorname{argmin}_{\boldsymbol{\theta}} L(\boldsymbol{\theta} ; \mathbf{I}, \mathbf{R})$ can be learnt by applying standard gradient based optimizer (e.g., BFGS) onto the loss function under the constraint that $\beta, \sigma, \gamma, \mu \in[0,1]$.

$$
L(\boldsymbol{\theta} ; \mathbf{I}, \mathbf{R})=\frac{1}{T} \sum_{t=1}^T\left[\left(\log \left(\widehat{I}_t+p\right)-\log \left(I_t+p\right)\right)^2+\left(\log \left(\widehat{R}_t+p\right)-\log \left(R_t+p\right)\right)^2\right],
$$


In [166]:
def loss(pred, target, smoothing=20): 
    return np.mean((np.log(pred+smoothing) - np.log(target+smoothing))**2)
    
# SEIR: the 'reported' SEIR. In our case, the simulated dataset. 
def optim(SEIR):
    # maybe these could be passed into the function as  
    # parameters if the estimation parts change
    y0 = [800.0, 100.0, 50.0, 50.0]
    t = np.linspace(0, 59, 60)
    N  = 1000.0  
    def deriv(y, t, N, beta, mu, sigma, gamma):
        s,e,i,r= y
        dsdt = -((beta * (e + i) * s) / N)
        dedt = (((beta * (e + i) * s) / N) - sigma * e)
        didt = (mu * sigma * e - (gamma * i))
        drdt = (gamma * i)  
        return dsdt, dedt, didt, drdt
    
    def objective(x):
        beta, mu, sigma, gamma = x
        est_SEIR = odeint(deriv, y0, t, args=(N, beta, mu, sigma, gamma))
        est_df = pd.DataFrame(est_SEIR, columns = ['S','E','I','R'])
        return loss(est_df.loc[:,"I"], SEIR.loc[:,"I"]) +  loss(est_df.loc[:,"R"], SEIR.loc[:,"R"])

    # scipy optimizer
    optimal = minimize(
        objective,
        [0.05,0.05,0.05,0.05], # initial estimate
        method='L-BFGS-B',
        bounds=[(0.0001, .3), (0.001, 0.3), (0.01, 1), (0.001, 1.)] # bounded based on prior knowledge
    )

    return optimal.x

Load datasets and then minimize the objective function and obtain parameter estimates:

In [167]:
os.chdir('c:\\Users\\chloe\\Desktop\\ChloeYou\\UBC_grad\\Term3\\MATH561\\math561-gp-SuEIR\\data\\python')

params_trad = pd.DataFrame([])
for file in os.listdir(os.getcwd()) :
    df = pd.read_csv(file)
    params_trad[file] = optim(df)
    
params_trad # rows: beta, mu, sigma, gamma


,sim-1-p.csv,sim-10-p.csv,sim-2-p.csv,sim-3-p.csv,sim-4-p.csv,sim-5-p.csv,sim-6-p.csv,sim-7-p.csv,sim-8-p.csv,sim-9-p.csv
0,0.060931,0.178104,0.199082,0.153612,0.232099,0.124057,0.300000,0.300000,0.244830,0.181164
1,0.300000,0.300000,0.300000,0.300000,0.134965,0.300000,0.113158,0.114356,0.297886,0.300000
2,0.225498,0.051532,0.055540,0.057686,0.124354,0.078436,0.183636,0.193247,0.028638,0.051798
3,0.121821,0.098759,0.083482,0.113969,0.121669,0.120308,0.125316,0.126875,0.121924,0.095446


In [168]:
os.chdir('c:\\Users\\chloe\\Desktop\\ChloeYou\\UBC_grad\\Term3\\MATH561\\math561-gp-SuEIR\\data\\matlab')

csv = os.listdir(os.getcwd())
del csv[0] # this is the ground truth so we will only calculate the R_0 from it
print(csv)
params_nn = pd.DataFrame([])
for file in csv :
    df = pd.read_csv(file)
    params_nn[file] = optim(df)
    
params_nn # rows: beta, mu, sigma, gamma

['sim-1-m.csv', 'sim-10-m.csv', 'sim-2-m.csv', 'sim-3-m.csv', 'sim-4-m.csv', 'sim-5-m.csv', 'sim-6-m.csv', 'sim-7-m.csv', 'sim-8-m.csv', 'sim-9-m.csv']


,sim-1-m.csv,sim-10-m.csv,sim-2-m.csv,sim-3-m.csv,sim-4-m.csv,sim-5-m.csv,sim-6-m.csv,sim-7-m.csv,sim-8-m.csv,sim-9-m.csv
0,0.300000,0.100088,0.300000,0.299920,0.199981,0.100091,0.300000,0.299559,0.299836,0.200036
1,0.060381,0.099935,0.055187,0.050007,0.050002,0.049961,0.118024,0.109000,0.100018,0.099986
2,0.163390,0.090045,0.129959,0.090023,0.089998,0.090034,0.166595,0.131456,0.090065,0.089980
3,0.119317,0.120001,0.119582,0.120002,0.119997,0.120000,0.119265,0.119599,0.120000,0.120000


Calculate R_0 for each simulation:
$$
R_0 = \frac{\beta}{\sigma} +\frac{\beta\mu}{\gamma}
$$

In [169]:
R_0_trad = (params_trad.loc[0,]/params_trad.loc[2,]) + (params_trad.loc[0,]*params_trad.loc[1,]/params_trad.loc[3,])
R_0_trad


sim-1-p.csv     0.420256
sim-10-p.csv    3.997187
sim-2-p.csv     4.299878
sim-3-p.csv     3.067265
sim-4-p.csv     2.123900
sim-5-p.csv     1.890971
sim-6-p.csv     1.904559
sim-7-p.csv     1.822815
sim-8-p.csv     9.147381
sim-9-p.csv     4.066956
dtype: float64

In [170]:
R_0_nn = (params_nn.loc[0,]/params_nn.loc[2,]) + (params_nn.loc[0,]*params_nn.loc[1,]/params_nn.loc[3,])
R_0_nn

sim-1-m.csv     1.987912
sim-10-m.csv    1.194881
sim-2-m.csv     2.446875
sim-3-m.csv     3.456575
sim-4-m.csv     2.305388
sim-5-m.csv     1.153372
sim-6-m.csv     2.097651
sim-7-m.csv     2.551799
sim-8-m.csv     3.579027
sim-9-m.csv     2.389792
dtype: float64

Let's compare these estimated values to the 'true' $R_0 = \frac{0.1}{0.09} +\frac{0.1*0.1}{0.12} = 1.19$ 

Also, since the objective function is non-convex, the initial parameter used for optimization
varies the estimate a lot. 

We look at the sum of the absolute value of bias:

In [174]:
sum(abs(R_0_trad - 1.19))

22.380655298428383

In [175]:

sum(abs(R_0_nn - 1.19))

11.336527242462807